# LastStickerStat и LastStickerBot

Класс **LastStickerStat** - это инструмент для парсинга и обработки данных по лотам сайта коллекционеров [laststicker.ru](https://www.laststicker.ru/)
Он может работать как самостоятельно, так и в "тандеме" с телеграм ботом. Последнее не требует у пользователя програмистских навыков и позволяет обновлять данные и делать запросы к их базе, ограничиваясь командами бота и интуитивно понятным "языком" фильтрации, разработанным специально под эти цели.

*Зачем оно нужно и что дает?*

К сожалению, у [laststicker.ru](https://www.laststicker.ru/) нет большого архива лотов (есть только архив лотов, которые закрылись за последний месяц) и информации о продажах (кто и кому продал определенный предмет, когда, за сколько и т. д.).

К счастью, старые лоты (недоступные через интерфейс) можно открывать по ссылкам вида `www.laststicker.ru/auction/post{id}`, что и дает возможность парсинга. 

Поэтому доступ к этой большой базе (насчитывающей более 400К лотов) и поиску по ней дает:
- возможность найти информацию о том, кому была продана определенная (желанная пользователю) вещь (коллекционный журнал, карточка, наклейка или их блок и т. д.) и договориться о перекупе
- возможность анализировать информацию о тенденции цен по коллекциям/предметам/... (если пользователь занимается коллекционным бизнесом или инвестированием в эту сферу с целью долгосрочного дохода)
- понимание, как общаться с клиентом/продавцом на сделке (можно ли торговаться, например), учитывая то, по каким ценам он покупал/продавал предметы ранее
- делать разные выводы, основываясь на статистических данных, и многое другое!

In [1]:
from LastStickerStat import *

Пусть база уже существует и находится в папке *data* или базы еще нет, и мы хотим, чтобы она загрузилась в эту папку (очень желательно, чтобы она была пустая, т.к. если в дальнейшем захочется запустить бота на этих данных, то пользователи со *специальным доступом* смогут выгружать архив всей папки).  
*Убедитесь, что в папке с LastStickerStat.py находится `config.json`. Он необходим, как минимум, для правильной работы парсинга. Изменения данных этого файла может привести к непредвиденным последствиям!  

(То же самое можно сделать, **запустив скрипт** `python3 LastStickerStat.py data`, только это имеет смысл, если мы создаем базу с нуля)

**Имейте в виду, что если базы еще не существует, то все 400К+ лотов будут загружаться 15-25 часов в зависимости от скорости вашего интернета**  

Создаем объект класса `LastStickerStat`:

In [2]:
stat = LastStickerStat("data")

Для удобства я добавила графический ползунок процента загрузки. Если базы не было или мы специально создаем ее по новой, будет 2 ползунка: первый информирует о проценте загрузки данных "старого" формата, второй - "нового".  
Я их так назвала просто потому, что лоты до, примерно, 2014 года выглядят (а главное, размечены в html) одним образом, а после - другим. Пример ["старого"](https://www.laststicker.ru/blog/post10112/) (ставки делаются в комментариях, без ML, кажется, в основном нельзя понять, выкупили ли лот, если да, кто и по какой цене) формата и ["нового"](https://www.laststicker.ru/auction/post484105/) (у таких страниц +- строгая разметка и достаточно легко выпарсить, что нужно).  
Староформатные лоты помечаются в базе флажком "Староформатный" в колонке lot_type и кроме названия лота, дат открытия/закрытия лота, информации о продавце и коллекции никакой особо содержательной информации больше не записывается. Т.к. лотов такого типа относительно мало и в основном база создавалась под нужды определенного человека, которого устроила такая ситуация, то и меня это устроило.

Теперь, когда обьект `stat` инициализирован, у него есть несколько информационных атрибутов, в числе которых:

In [3]:
print(stat.csv_path) # путь к нашей базе в csv формате
print(stat.upper_csv_path) # путь к "базе верхних регистров"; не несет никакой новой информации, создана для
                           # более быстрого и простого поиска лотов по подстрокам
print(stat.info["last_lot_parse_date"]) # дата и время последнего обновления базы
stat.df # DataFrame наших данных

data/data.csv
data/upper_data.csv
10:5 28.1.2022


,title,url,subject,theme,lot_type,status,is_bet_made,collection,url_collection,date_start_day,...,seller_location_district,seller_location_country,initial_price,last_price,last_price_author,url_last_buyer,date_end_day,date_end_month,date_end_year,date_end_time
0,черепашки ниндзя (часть 1-ая из 2-х),https://www.laststicker.ru/auction/post3982/,Неопределено,Мультфильмы и кино,Староформатный,Закрыт,NaN,NaN,NaN,17,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Полный комплект наклеек Panini NHL 95-96. 306 ...,https://www.laststicker.ru/auction/post3983/,Наклейки,Хоккейные наклейки,Староформатный,Закрыт,NaN,PANINI NHL 1995-1996,https://www.laststicker.ru/cards/panini_nhl_95...,17,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Panini Пикси / Pixie,https://www.laststicker.ru/auction/post3984/,Наклейки,Мультфильмы и кино,Староформатный,Закрыт,NaN,PANINI Пикси,https://www.laststicker.ru/cards/panini_pixie/,18,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Продаётся на 100% заполненный альбом Панда Кун...,https://www.laststicker.ru/auction/post3985/,Наклейки,Мультфильмы и кино,Староформатный,Закрыт,NaN,АСТ Кунг-Фу Панда,https://www.laststicker.ru/cards/ast_kung_fu_p...,18,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Куплю любое количество стикеров РФПЛ 2009 в пр...,https://www.laststicker.ru/auction/post3987/,Наклейки,Футбольные наклейки,Староформатный,Закрыт,NaN,Российская Футбольная Премьер-Лига 2009,https://www.laststicker.ru/cards/sportssticker...,18,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
474893,Почти пустой альбом Panini Утиные истории 1987,https://www.laststicker.ru/auction/post484168/,Наклейки,Мультфильмы и кино,Аукцион,Открыт,False,PANINI Утиные истории,https://www.laststicker.ru/cards/panini_duck_t...,28,...,Забайкальский край,Россия,699.0,NaN,NaN,NaN,2.0,2.0,2022.0,15:00
474894,Альбом Mickey Story 1978,https://www.laststicker.ru/auction/post484169/,Наклейки,Мультфильмы и кино,Аукцион,Открыт,False,PANINI Mickey Story,https://www.laststicker.ru/cards/panini_mickey...,28,...,Забайкальский край,Россия,699.0,NaN,NaN,NaN,1.0,2.0,2022.0,15:00
474895,12 разных наклеек по коллекции Panini Белоснеж...,https://www.laststicker.ru/auction/post484170/,Наклейки,Мультфильмы и кино,Аукцион,Открыт,False,PANINI Белоснежка и 7 гномов,https://www.laststicker.ru/cards/panini_snow_w...,28,...,Забайкальский край,Россия,180.0,NaN,NaN,NaN,1.0,2.0,2022.0,15:00
474896,Продажа 206 стикеров без повторов по коллекции...,https://www.laststicker.ru/auction/post484171/,Наклейки,Хоккейные наклейки,Аукцион,Открыт,False,SeReal КХЛ 2011-2012,https://www.laststicker.ru/cards/sereal_khl_20...,28,...,Владимирская область,Россия,150.0,NaN,NaN,NaN,4.0,2.0,2022.0,09:48


Теперь таблицу можно анализировать руками с помощью функционала pandas. Для начала посмотрим, какими вообще могут быть значения колонок с ограниченным набором значений

In [4]:
print("subject = ", stat.df.subject.unique())
print("\ntheme = ", stat.df.theme.unique())
print("\nlot_type = ", stat.df.lot_type.unique())
print("\nstatus = ", stat.df.status.unique())
print("\nis_bet_made = ", stat.df.is_bet_made.unique())

subject =  ['Неопределено' 'Наклейки' 'Карточки' 'Другое']

theme =  ['Мультфильмы и кино' 'Хоккейные наклейки' 'Футбольные наклейки' 'ККИ'
 'Прочие наклейки' 'Прочее' 'Другие виды спорта' 'Аксессуары'
 'Совместные заказы' 'Футбольные карточки' 'Хоккейные карточки'
 'Прочие карточки' 'Монеты, купюры' 'Значки' 'Марки' 'Печатная продукция'
 'Модели']

lot_type =  ['Староформатный' 'Аукцион' 'Объявление']

status =  ['Закрыт' 'Открыт']

is_bet_made =  [nan True False]


Остальные колонки, как можно видеть из `DataFrame`, это либо строки, либо числа (либо `NaN`). У каждого лота свои  Вообще говоря, названия колонок достаточно хорошо передают их суть, поэтому должно быть понятно, какие данные будут в каждом столбце  

Функционал объекта класса `LastStickerStat`, который не "привязан" к определенному боту, довольно ограничен. Можно руками получить любой желаемый срез данных, например:

In [5]:
# Ищет карточные лоты со ставками, в названии коллекции которых есть подстрока Поттер 
df = stat.df[(stat.df.collection.str.contains("Поттер") == True) & (stat.df.subject == "Карточки") & stat.df.is_bet_made == True]
df

,title,url,subject,theme,lot_type,status,is_bet_made,collection,url_collection,date_start_day,...,seller_location_district,seller_location_country,initial_price,last_price,last_price_author,url_last_buyer,date_end_day,date_end_month,date_end_year,date_end_time
116573,Полный сет карточек - 81 штука,https://www.laststicker.ru/auction/post123607/,Карточки,Мультфильмы и кино,Аукцион,Закрыт,True,WIZARDS Гарри Поттер и Философский Камень - Ки...,https://www.laststicker.ru/cards/wizards_harry...,10,...,Нижегородская область,Россия,500.0,580.0,VitVit,https://www.laststicker.ru/users/36590/,11.0,11.0,2015.0,22:22
116574,Полный сет карточек - 89 штук,https://www.laststicker.ru/auction/post123608/,Карточки,Мультфильмы и кино,Аукцион,Закрыт,True,CARDS Гарри Поттер и Узник Азкабана,https://www.laststicker.ru/cards/cards_harry_p...,10,...,Нижегородская область,Россия,500.0,1050.0,cergei-75,https://www.laststicker.ru/users/51910/,11.0,11.0,2015.0,22:41
168239,Пустой Биндер Гарри Поттер и Узник Азкабана,https://www.laststicker.ru/auction/post175677/,Карточки,Мультфильмы и кино,Аукцион,Закрыт,True,ARTBOX Гарри Поттер и Узник Азкабана (1 часть),https://www.laststicker.ru/cards/artbox_harry_...,2,...,Московская область,Россия,400.0,420.0,Артур М,https://www.laststicker.ru/users/57773/,4.0,1.0,2017.0,22:50
183023,Гарри Поттер и Философский Камень. Кинокарточки,https://www.laststicker.ru/auction/post190557/,Карточки,Мультфильмы и кино,Аукцион,Закрыт,True,WIZARDS Гарри Поттер и Философский Камень - Ки...,https://www.laststicker.ru/cards/wizards_harry...,7,...,Московская область,Россия,790.0,830.0,Tanya-Tanya,https://www.laststicker.ru/users/60216/,14.0,4.0,2017.0,21:37
232617,Wizards Гарри Поттер. ККИ (Quidditch Cup Set) ...,https://www.laststicker.ru/auction/post240433/,Карточки,Мультфильмы и кино,Аукцион,Закрыт,True,WIZARDS Гарри Поттер. ККИ (Quidditch Cup Set),https://www.laststicker.ru/cards/wizards_harry...,1,...,Татарстан,Россия,80.0,89.0,Shikrokami,https://www.laststicker.ru/users/41094/,8.0,4.0,2018.0,17:04
234027,Wizards Гарри Поттер. ККИ (Quidditch Cup Set) ...,https://www.laststicker.ru/auction/post241849/,Карточки,Мультфильмы и кино,Аукцион,Закрыт,True,WIZARDS Гарри Поттер. ККИ (Quidditch Cup Set),https://www.laststicker.ru/cards/wizards_harry...,12,...,Татарстан,Россия,80.0,84.0,ден супермен,https://www.laststicker.ru/users/60610/,13.0,4.0,2018.0,12:30
244626,Гарри Поттер и Философский Камень - Кинокарточки,https://www.laststicker.ru/auction/post252534/,Карточки,Мультфильмы и кино,Аукцион,Закрыт,True,WIZARDS Гарри Поттер и Философский Камень - Ки...,https://www.laststicker.ru/cards/wizards_harry...,28,...,Хабаровский край,Россия,500.0,500.0,NinaSon,https://www.laststicker.ru/users/61258/,4.0,7.0,2018.0,21:00
247016,Wizards Гарри Поттер и философский камень (ред...,https://www.laststicker.ru/auction/post254938/,Карточки,Мультфильмы и кино,Аукцион,Закрыт,True,WIZARDS Гарри Поттер и Философский Камень - Ки...,https://www.laststicker.ru/cards/wizards_harry...,12,...,Воронежская область,Россия,500.0,500.0,Dredd,https://www.laststicker.ru/users/38492/,18.0,7.0,2018.0,23:00
276162,15 пакетиков panini Гарри Поттер и Дары смерти...,https://www.laststicker.ru/auction/post284290/,Карточки,Мультфильмы и кино,Аукцион,Закрыт,True,ARTBOX Гарри Поттер и Дары Смерти. Часть 1,https://www.laststicker.ru/cards/artbox_harry_...,12,...,Московская область,Россия,333.0,333.0,papa_john,https://www.laststicker.ru/users/291/,15.0,1.0,2019.0,20:57
285222,Пустой альбом 2 шт,https://www.laststicker.ru/auction/post293404/,Карточки,Мультфильмы и кино,Аукцион,Закрыт,True,ARTBOX Гарри Поттер и Дары Смерти. Часть 2,https://www.laststicker.ru/cards/artbox_harry_...,28,...,Тверская область,Россия,50.0,460.0,Sanchel,https://www.laststicker.ru/users/31201/,4.0,3.0,2019.0,18:01


В базе есть много карточек и наклеек фирмы Panini, но в разных лотах она может быть написана в разных регистрах. Найдем все лоты, которые имеют отношение к этой фирме, используя вспомогательную таблицу верхних регистров `stat.df_upper`

In [6]:
word = "Panini"
stat.df.loc[stat.df_upper[(stat.df_upper.collection.str.contains(word.upper()) == True) | (stat.df_upper.title.str.contains(word.upper()) == True)].index]

,title,url,subject,theme,lot_type,status,is_bet_made,collection,url_collection,date_start_day,...,seller_location_district,seller_location_country,initial_price,last_price,last_price_author,url_last_buyer,date_end_day,date_end_month,date_end_year,date_end_time
1,Полный комплект наклеек Panini NHL 95-96. 306 ...,https://www.laststicker.ru/auction/post3983/,Наклейки,Хоккейные наклейки,Староформатный,Закрыт,NaN,PANINI NHL 1995-1996,https://www.laststicker.ru/cards/panini_nhl_95...,17,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Panini Пикси / Pixie,https://www.laststicker.ru/auction/post3984/,Наклейки,Мультфильмы и кино,Староформатный,Закрыт,NaN,PANINI Пикси,https://www.laststicker.ru/cards/panini_pixie/,18,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Евро-96 собранный на 96%.,https://www.laststicker.ru/auction/post3993/,Наклейки,Футбольные наклейки,Староформатный,Закрыт,NaN,PANINI Чемпионат Европы 1996,https://www.laststicker.ru/cards/panini_euro_1...,18,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Куплю Альбом+2 блока World Cup STORY,https://www.laststicker.ru/auction/post4003/,Наклейки,Футбольные наклейки,Староформатный,Закрыт,NaN,PANINI История Чемпионатов Мира,https://www.laststicker.ru/cards/panini_wc_story/,19,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Чемпионат Мира 1998 (Pаnini),https://www.laststicker.ru/auction/post4004/,Наклейки,Футбольные наклейки,Староформатный,Закрыт,NaN,PANINI Чемпионат Мира 1998,https://www.laststicker.ru/cards/panini_wc_1998/,19,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
474892,Заполненный альбом Panini Король Лев,https://www.laststicker.ru/auction/post484167/,Наклейки,Мультфильмы и кино,Аукцион,Открыт,False,PANINI Король Лев,https://www.laststicker.ru/cards/panini_lion_k...,28,...,Забайкальский край,Россия,3200.0,NaN,NaN,NaN,2.0,2.0,2022.0,15:00
474893,Почти пустой альбом Panini Утиные истории 1987,https://www.laststicker.ru/auction/post484168/,Наклейки,Мультфильмы и кино,Аукцион,Открыт,False,PANINI Утиные истории,https://www.laststicker.ru/cards/panini_duck_t...,28,...,Забайкальский край,Россия,699.0,NaN,NaN,NaN,2.0,2.0,2022.0,15:00
474894,Альбом Mickey Story 1978,https://www.laststicker.ru/auction/post484169/,Наклейки,Мультфильмы и кино,Аукцион,Открыт,False,PANINI Mickey Story,https://www.laststicker.ru/cards/panini_mickey...,28,...,Забайкальский край,Россия,699.0,NaN,NaN,NaN,1.0,2.0,2022.0,15:00
474895,12 разных наклеек по коллекции Panini Белоснеж...,https://www.laststicker.ru/auction/post484170/,Наклейки,Мультфильмы и кино,Аукцион,Открыт,False,PANINI Белоснежка и 7 гномов,https://www.laststicker.ru/cards/panini_snow_w...,28,...,Забайкальский край,Россия,180.0,NaN,NaN,NaN,1.0,2.0,2022.0,15:00


Еще можно загрузить результаты поиска в *csv*, открыть в Excel (LibreOffice и др.) и просмотреть найденные лоты глазами

In [39]:
path = "filtered_{}.csv".format(stat.dir_path)
df.to_csv(path, index=False, sep=";") # результат поиска лотов по Поттеру
print("Отфильтрованные данные находятся в файле {}".format(path))

Отфильтрованные данные находятся в файле filtered_test9.csv


Также, можно обновлять данные в базе: менять статус лота с *Открыт* на *Закрыт*, если лот закрылся, подгружать новую информацию о ставках, покупателях и т.д., и загружать информацию о новых, недавно открывшихся лотах

In [41]:
date, url, lots = stat.update()
print("Данные актуальны на {}.\nПоследний загруженный лот - {}.\nВсего новых лотов : {}".format(date, url, lots))

100%|██████████| 2349/2349 [02:04<00:00, 18.92it/s] 
0it [00:00, ?it/s]

Данные актуальны на 2:8 28.1.2022.
Последний загруженный лот - https://www.laststicker.ru/auction/post484154/.
Всего новых лотов : 0


Это весь функционал, доступный для использования без телеграм бота. 

В данный момент бот крутится на Google Platform сервере и еще будет крутится пару месяцев, пока не закончится бесплатный период. Поэтому функционал бота можно потестировать двумя способами:  
**1)** Найти в телеграме [@LastStickerStatBot](https://t.me/LastStickerStatBot), нажать старт и дождаться пока я (мне прилетит ваш id) дам вам доступ (я могу раздавать доступ также через телеграм). Это делается нажатием одной команды, поэтому если я не буду спать в то время, как вы кликните /start, доступ у вас будет моментально.    

**2)** Работать со своим ботом, запустив скрипт `LastStickerBot.py` на своей машине, следуя инструкциям ниже

Узнайте свой пользовательский телеграм id и впишите его в переменную `tg_id`. Это можно сделать с помощью бота [@getmyid_bot](https://t.me/getmyid_bot)

In [5]:
tg_id = YOUR_ID_HERE

Занесем ваш id в список админов и разрешенных пользователей бота

In [6]:
with open("bot_config.json", "r") as f:
    bot_config = json.load(f)

bot_config["allowed_ids"].append(tg_id)
bot_config["ADMIN_IDS"].append(tg_id)
    
with open("bot_config.json", "w") as f:
    json.dump(bot_config, f)

Теперь запустите бота командой в терминале `python3 LastStickerBot.py data`  
Конечно, на месте `data`, вы пишете название папки, в которой лежит база, с которой вы хотите работать

Теперь откройте своего бота и наберите команду `/start` или `/help`.  
Я постаралась подробно и четко расписать инструкции по работе с ботом, поэтому все, что может понадобиться для ознакомления, доступно по командам `/help` и `/howfilter`

**О преимуществах бота вкраце:**

- работа с базой с любых устройств, на которые можно установить Telegram
- одновременно несколько человек могут взаимодействовать с базой данных
- фильтрация данных на около естественном языке по команде `/filter` (см. `/howfilter`)
- создание **пользовательских фильтров**: если у вас есть фильтр, по которому вы часто отфильтровываете данные (например, вы часто ищете лоты по определенной коллекции карточек), вы можете сохранить его в базу пользовательских фильтров, придумав ему название (подробнее по команде `/newfilter`) 
- можно не хранить всю огромную таблицу на локальной машине, при этом продолжая с ней работать; при необходимости можно скачать ее в *csv* формате или (только для админов) вообще всю информацию о базе, включая информацию о id разрешенных пользователей, их пользовательских фильтрах и т. д. 

**Additional information:**

Мой бот [@LastStickerStatBot](https://t.me/LastStickerStatBot) пока работает напрямую с Telegram API Server, поэтому имеет ограничение на отправку файлов в размере 50 МБ (не более).  
Учитывая тот факт, что, если пользователь просит выгрузить файл превосходящий этот лимит, бот высылает архив сильно меньшего размера, то пока это ограничение сильно не мешает. Тем более адекватные запросы (на несколько пересечений, например), которые в большинстве своем и делаются, находят относильно мало (несколько сотен) лотов, а размер файла с таким количеством записей далек от лимита.  
Если это ограничение начнет сильно мешать, установлю Local API Server